In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()
from pyspark.ml.feature import Tokenizer, HashingTF, IDF

In [2]:
sentenceData = spark.createDataFrame([
    (0.0, "Hi I heard about Sparkl"),
    (0.0, "I wish Java could use case Classes"),
    (1.0, "Logistic regression models are neat")
], ["label", "sentence"])

In [4]:
sentenceData.show(truncate=False)

+-----+-----------------------------------+
|label|sentence                           |
+-----+-----------------------------------+
|0.0  |Hi I heard about Sparkl            |
|0.0  |I wish Java could use case Classes |
|1.0  |Logistic regression models are neat|
+-----+-----------------------------------+



In [5]:
tokenizer = Tokenizer(inputCol='sentence', outputCol='words')

In [6]:
words_data = tokenizer.transform(sentenceData)

In [7]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Sparkl            |[hi, i, heard, about, sparkl]             |
|0.0  |I wish Java could use case Classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [8]:
hashing_tf = HashingTF(inputCol='words', outputCol='rawFeatures')

In [9]:
featurized_data = hashing_tf.transform(words_data)

In [10]:
idf = IDF(inputCol='rawFeatures', outputCol='features')

In [11]:
idf_model = idf.fit(featurized_data)

In [12]:
rescaled_data = idf_model.transform(featurized_data)

In [14]:
rescaled_data.select('label', 'features').show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,116355],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

In [15]:
from pyspark.ml.feature import CountVectorizer

In [16]:
df = spark.createDataFrame([
    (0, "a,b,c".split(" ")),
    (1, "a b b c a".split(" "))
], ["id", "words"])

In [17]:
df.show(truncate=False)

+---+---------------+
|id |words          |
+---+---------------+
|0  |[a,b,c]        |
|1  |[a, b, b, c, a]|
+---+---------------+



In [22]:
cv = CountVectorizer(inputCol='words', outputCol='features', vocabSize=3, minDF=2.0)

In [23]:
model = cv.fit(df)

In [24]:
result = model.transform(df)

In [25]:
result.show(truncate=False)

+---+---------------+---------+
|id |words          |features |
+---+---------------+---------+
|0  |[a,b,c]        |(0,[],[])|
|1  |[a, b, b, c, a]|(0,[],[])|
+---+---------------+---------+

